In [1]:
!pip install deeptrack

<!--<badge>--><a href="https://colab.research.google.com/github/softmatterlab/DeepTrack-2.0/blob/develop/examples/module-examples/image_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><!--</badge>-->

# deeptrack.image

This example introduces the module deeptrack.image.

## 1. What is an Image?

An `Image` is a container used by a feature to store both the generated image and the properties used to generate it. It is a subclass of numpy `ndarray`, so that any operation that works for `ndarrays` will also work for `Image`. `Image` has also a field `properties`, which contains the information about how the image has been generated (see also [properties_example](properties_example.ipynb)). Specifically, `properties` is a list of dictionaries, where each dictionary holds the current values of the properties of a feature (with the names of the properties as keys). This list is ordered as the features have been resolved.

By storing the properties used to resolve the image, information about the image can be accessed without access to the feature series. This allows features to change their behaviour depending on what is already in the image. This also allows to extract numeric information about the image (e.g., to be used in supervised learning). 

In [2]:
import numpy as np

from deeptrack.features import Feature
from deeptrack.image import Image

class Particle(Feature):
    def get(self, image, position=None, **kwargs):
        
        # Code for simulating a particle not included
        
        return image
    
image_shape = (256, 256)
    
particle = Particle(
    position=lambda: np.random.rand(2) * np.array(image_shape)
)

input_image = Image(np.zeros(image_shape))

output_image = particle.resolve(input_image)
output_image.properties

[{'position': array([168.84923801,  53.09129026]), 'name': 'Particle'}]

Beyond the expected properties, `name` and `position`, we also see a property `hash_key`. This is a special property injected by all features and is used to compare dictionaries of properties. This is especially useful when merging properties from different Images, where you don't want to duplicate some property. This can be done using the method `merge_properties_from(other)`

In [3]:
second_image = particle.resolve(input_image)

output_image.merge_properties_from(second_image)
output_image.properties

[{'position': array([168.84923801,  53.09129026]), 'name': 'Particle'}]

However if we update the feature between merges, the properties are treated as distinct. This is true even if the properties are not randomized.

In [4]:
particle.update()

updated_image = particle.resolve(input_image)

output_image.merge_properties_from(updated_image)
output_image.properties

[{'position': array([168.84923801,  53.09129026]), 'name': 'Particle'},
 {'position': array([  4.65292344, 140.98926194]), 'name': 'Particle'}]

## 2. Extracting the information with get_property

To retrieve information from the image, one can iterate through the list and extract the properties needed. The function `.get_property()` is a shorthand for this. It retrieves a property or a list of properties from an image. By default it retrieves a single instance of a property (the first instance that it finds).

In [5]:
output_image.get_property("position")

array([168.84923801,  53.09129026])

To retrieve all instances of a property, set `get_one` to `False`

In [6]:
output_image.get_property("position", get_one=False)

[array([168.84923801,  53.09129026]), array([  4.65292344, 140.98926194])]

Finally, if the property is not found, `default` is returned instead 

In [7]:
output_image.get_property("a_property_that_does_not_exist", default="Not found!")

'Not found!'

## 3. Storing metadata about an image not used by the feature

Sometimes it's convenient to store some information about a feature beyond what is strictly necessary to generate it. For example, you may want to identify one particle to track, or you might want to store the randomized diffusion constant of a particle used to generate physically-accurate sequences of images. This can be done by passing additional keyword arguments to the constructor of a feature. These will be stored as properties just like any other input to the constructor.

In [8]:
particle_to_track = Particle(
    position=lambda: np.random.rand(2) * np.array(image_shape),
    track_me=True
)

particle_not_to_track = Particle(
    position=lambda: np.random.rand(2) * np.array(image_shape),
    track_me=False
)

input_image = Image(np.zeros(image_shape))

particles = (particle_not_to_track^4) >> particle_to_track

output_image = particles.resolve(input_image, metadata='This is some additional data')
output_image.get_property("track_me", get_one=False)

[False, False, False, False, True]

In [9]:
particle = Particle(
    position=lambda: np.random.rand(2) * np.array(image_shape),
    diffusion=lambda: 1 + np.random.rand() * 4
) 

input_image = Image(np.zeros(image_shape))

particles = particle^3

particles.update()
output_image = particles.resolve(input_image)
output_image.get_property("diffusion", get_one=False)

[1.5869123041979534, 3.5725040600553326, 4.145845402448723]